In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import flwr
import torch
import dotenv
import wandb

from torch.optim.lr_scheduler import CosineAnnealingLR

from fl_g13.architectures import BaseDino
from fl_g13.editing import SparseSGDM

from fl_g13.fl_pytorch import build_fl_dependencies

from fl_g13.fl_pytorch.datasets import reset_partition
from fl_g13.modeling import load_or_create
from fl_g13.editing import SparseSGDM
from torch.optim import SGD

from fl_g13.fl_pytorch import get_client_app, get_server_app
from flwr.simulation import run_simulation

c:\Users\ciovi\Desktop\coding\mldl\fl-g13\.venv\lib\site-packages\dockerpycreds\utils.py:1: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils.spawn
2025-06-28 14:45:53.506 | INFO     | fl_g13.config:<module>:11 - PROJ_ROOT path is: C:\Users\ciovi\Desktop\coding\mldl\fl-g13


In [3]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Training on {DEVICE}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")

dotenv.load_dotenv()
build_fl_dependencies()

if DEVICE == "cuda":
    backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 1.0}}
else:
    backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 0.0}}

Training on cuda
Flower 1.17.0 / PyTorch 2.7.1+cu128
'vision_transformer.py' already exists.
'utils.py' already exists.


In [4]:
# login by key in .env file
WANDB_API_KEY = dotenv.dotenv_values()["WANDB_API_KEY"]
wandb.login(key=WANDB_API_KEY)

# Load checkpoint from .env file
CHECKPOINT_DIR = dotenv.dotenv_values()["CHECKPOINT_DIR"]

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\ciovi\_netrc
wandb: Currently logged in as: tarantino-giovanbattista01 (stefano-gamba-social-politecnico-di-torino) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
# Model config
## Model Hyper-parameters
head_layers = 3
head_hidden_size = 512
dropout_rate = 0.0
unfreeze_blocks = 12

## Training Hyper-parameters
batch_size = 128
lr = 1e-3
momentum = 0.9
weight_decay = 1e-5
T_max = 8
eta_min = 1e-5

# FL config
K = 100
C = 0.1
J = 8
num_shards_per_partition = 5 # Nc
partition_type = 'shard'

num_rounds = 100

## Server App config
save_every = 5
evaluate_each = 5
fraction_fit = C        # 0.1
fraction_evaluate = C   # 0.1
min_fit_clients = 10
min_evaluate_clients = 5
min_available_clients = 10

# model editing config
model_editing = True
mask_type = 'global'
sparsity = 0.9
calibration_rounds = 3
model_editing_batch_size = 1
mask = None

## simulation run config
NUM_CLIENTS = 100
MAX_PARALLEL_CLIENTS = 10

## Base model location
# The 200-epoch model folder
# Ensure that the most recent file is the correct one
model_save_path = CHECKPOINT_DIR + f"/fl/non-iid/{num_shards_per_partition}_{J}"

In [6]:
# Run simulations
reset_partition()

# print('-' * 200)
# print(f" Nc={num_shards_per_partition}, J={J}, mask_type={mask_type}, sparsity={sparsity}, mask_calibration_round={calibration_rounds}\n")

model_checkpoint = CHECKPOINT_DIR + f"/fl/non-iid/Baseline/{num_shards_per_partition}_{J}_{mask_type}_{sparsity}_{calibration_rounds}"

# Load Base model
model, start_epoch = load_or_create(
    path=model_save_path,
    model_class=BaseDino,
    model_config=None,
    optimizer=None,
    scheduler=None,
    device=DEVICE,
    verbose=True,
)
model.to(DEVICE)

if model_editing:
    # Create a dummy mask for SparseSGDM
    dummy_mask = [torch.ones_like(p, device=p.device) for p in model.parameters()]  
    
    optimizer = SparseSGDM(
        model.parameters(),
        mask=dummy_mask,
        lr=lr,
        momentum=momentum,
        weight_decay=weight_decay
    )
else:
    optimizer = SGD(model.parameters(), lr=lr, momentum=momentum)
    
criterion = torch.nn.CrossEntropyLoss()
scheduler = CosineAnnealingLR(
    optimizer=optimizer,
    T_max=T_max,
    eta_min=eta_min
)

# Unfreeze entire model for model_editing
model.unfreeze_blocks(unfreeze_blocks)

# Wandb settings
use_wandb = True
run_name = f"fl_bl_{num_shards_per_partition}_{J}_{mask_type}_{sparsity}_{calibration_rounds}"
wandb_config = {
    # wandb param
    'name': run_name,
    'project_name': f"fl_v5_{num_shards_per_partition}_{J}_baseline",
    'run_id': run_name,
    
    # fl config
    "fraction_fit": fraction_fit,
    "lr": lr,
    "momentum": momentum,
    'weight_decay': weight_decay,
    'partition_type': partition_type,
    'K': K,
    'C': C,
    'J': J,
    'Nc': num_shards_per_partition,
    
    # model config
    'head_layers': head_layers,
    'head_hidden_size': head_hidden_size,
    'dropout_rate': dropout_rate,
    'unfreeze_blocks': unfreeze_blocks,
    
    # model editing config
    'model_editing_batch_size': model_editing_batch_size,
    'mask_calibration_round': calibration_rounds,
    'mask_type': mask_type,
    'sparsity': sparsity
}

client = get_client_app(
    model=model,
    optimizer=optimizer,
    criterion=criterion,
    device=DEVICE,
    partition_type=partition_type,
    local_epochs=1,
    local_steps=J,
    batch_size=batch_size,
    num_shards_per_partition=num_shards_per_partition,
    scheduler=scheduler,
    model_editing=model_editing,
    mask_type=mask_type,
    sparsity=sparsity,
    mask=mask,
    model_editing_batch_size=model_editing_batch_size,
    mask_func=None,
    mask_calibration_round=calibration_rounds
)

server = get_server_app(
    checkpoint_dir=model_checkpoint,
    model_class=model,
    optimizer=optimizer,
    criterion=criterion,
    scheduler=scheduler,
    num_rounds=num_rounds,
    fraction_fit=fraction_fit,
    fraction_evaluate=fraction_evaluate,
    min_fit_clients=min_fit_clients,
    min_evaluate_clients=min_evaluate_clients,
    min_available_clients=min_available_clients,
    device=DEVICE,
    use_wandb=use_wandb,
    wandb_config=wandb_config,
    save_every=save_every,
    prefix='baseline',
    evaluate_each=evaluate_each,
    model= model,
    start_epoch= start_epoch
)

# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_CLIENTS,
    backend_config=backend_config
)

if use_wandb:
    wandb.finish()

🔍 Loading checkpoint from /Users/ciovi/Desktop/coding/mldl/fl-g13/checkpoints/fl/non-iid/1_8\1_8_200_epoch.pth
📦 Model class in checkpoint: BaseDino
🔧 Model configuration: {'variant': 'dino_vits16', 'dropout_rate': 0.0, 'head_hidden_size': 512, 'head_layers': 3, 'num_classes': 100, 'unfreeze_blocks': 0, 'activation_fn': 'GELU', 'pretrained': True}


Using cache found in C:\Users\ciovi/.cache\torch\hub\facebookresearch_dino_main
Using cache found in C:\Users\ciovi/.cache\torch\hub\facebookresearch_dino_main


➡️ Moved model to device: cuda
✅ Loaded checkpoint from /Users/ciovi/Desktop/coding/mldl/fl-g13/checkpoints/fl/non-iid/1_8\1_8_200_epoch.pth, resuming at epoch 201
Using strategy 'CustomFedAvg' (default option)


c:\Users\ciovi\Desktop\coding\mldl\fl-g13\.venv\lib\site-packages\wandb\analytics\sentry.py:259: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}  # noqa


INFO :      Starting Flower ServerApp, config: num_rounds=100, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
Eval progress: 100%|██████████| 313/313 [00:24<00:00, 12.78batch/s]
INFO :      [Round 0] Centralized Evaluation - Loss: 2.4058, Metrics: {'centralized_accuracy': 0.3788}
INFO :      initial parameters (loss, other metrics): 2.405825910476831, {'centralized_accuracy': 0.3788}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 10 clients (out of 100)
(ClientAppActor pid=19820) 2025-06-28 14:46:29.615 | INFO     | fl_g13.config:<module>:11 - PROJ_ROOT path is: C:\Users\ciovi\Desktop\coding\mldl\fl-g13
(ClientAppActor pid=19820) c:\Users\ciovi\Desktop\coding\mldl\fl-g13\.venv\lib\site-packages\dockerpycreds\utils.py:1: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools 

...

💾 Saved checkpoint at: /Users/ciovi/Desktop/coding/mldl/fl-g13/checkpoints/fl/non-iid/Baseline/1_8_global_0.7_3\fl_baseline_BaseDino_epoch_295.pth


Eval progress: 100%|██████████| 313/313 [00:22<00:00, 13.87batch/s]
INFO :      [Round 95] Centralized Evaluation - Loss: 2.5036, Metrics: {'centralized_accuracy': 0.3652}
INFO :      fit progress: (95, 2.503641878454068, {'centralized_accuracy': 0.3652}, 9925.6301381)
INFO :      configure_evaluate: strategy sampled 10 clients (out of 100)
INFO :      aggregate_evaluate: received 10 results and 0 failures
INFO :      
INFO :      [ROUND 96]
INFO :      configure_fit: strategy sampled 10 clients (out of 100)
INFO :      aggregate_fit: received 10 results and 0 failures
INFO :      [Round 96] Avg Drift: 0.0009 | Relative Drift: 0.0000
INFO :      fit progress: (96, {}, {}, 9994.444187)
INFO :      configure_evaluate: strategy sampled 10 clients (out of 100)
INFO :      aggregate_evaluate: received 10 results and 0 failures
INFO :      
INFO :      [ROUND 97]
INFO :      configure_fit: strategy sampled 10 clients (out of 100)
INFO :      aggregate_fit: received 10 results and 0 failures


💾 Saved checkpoint at: /Users/ciovi/Desktop/coding/mldl/fl-g13/checkpoints/fl/non-iid/Baseline/1_8_global_0.7_3\fl_baseline_BaseDino_epoch_300.pth


Eval progress: 100%|██████████| 313/313 [00:22<00:00, 13.83batch/s]
INFO :      [Round 100] Centralized Evaluation - Loss: 2.2374, Metrics: {'centralized_accuracy': 0.4191}
INFO :      fit progress: (100, 2.237441005036473, {'centralized_accuracy': 0.4191}, 10292.0293066)
INFO :      configure_evaluate: strategy sampled 10 clients (out of 100)
INFO :      aggregate_evaluate: received 10 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 100 round(s) in 10308.98s
INFO :      	History (loss, distributed):
INFO :      		round 1: 2.5904916763305663
INFO :      		round 2: 2.848015570640564
INFO :      		round 3: 2.3292060494422913
INFO :      		round 4: 3.323385143280029
INFO :      		round 5: 3.0072263360023497
INFO :      		round 6: 2.5442407011985777
INFO :      		round 7: 2.842056429386139
INFO :      		round 8: 2.2210480690002443
INFO :      		round 9: 3.126421520113945
INFO :      		round 10: 2.1448901295661926
INFO :      		round 11: 2.1584503650665283

centralized_accuracy,▆▅▆▆▆▆▁▆▇▆▇▆▇▇▇▅▇▇█▆▇
centralized_eval_loss,▂▃▃▃▂▂█▃▂▂▁▂▂▂▁▄▂▂▁▃▁
decentralized_avg_eval_accuracy,▅▄▄▃▆▃▁▄▄▅▄▁▅█▆▇▂▄▆▃▄▅▅▆▄▅▃▆▄▇▆▄█▃▅▇▇▄▄▃
decentralized_avg_eval_loss,▄▅▃▇▆▃▆▅▄▄▃▅▄█▅▃▆▆▂▃▃▃▃▃▂▄▄▄▅▃▅▁▁▂▄▅▇▄▅▅
decentralized_avg_train_accuracy,▁▃▃▄▄▃▅▄▃▄▇▇▄▇▅▄▂▄▆▆▇▇▆▄▆▅▆▅▆▅▆█▆▇█▅▆▇▇█
decentralized_avg_train_loss,█▅▅▇█▇▅▆▅▄▆▆█▃▃▄▂▃▂▂▃▃▃▅▃▂▅▄▂▃▄▁▄▄▃▄▁▁▁▁
centralized_accuracy,0.4191
centralized_eval_loss,2.23744
decentralized_avg_eval_accuracy,0.294
decentralized_avg_eval_loss,2.82392
decentralized_avg_train_accuracy,0.90675
